In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a64eb2fe560084ff13f5bc2ab7ff0f74a11ffd3adc4694311f3b8b991d4f635c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType

### Домашнее задание:
**Условие:** создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

**Задание:**

— Используя Spark прочитайте данные из файла csv.

— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.

— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.

— Отсортируйте данные по общему объему продаж в порядке убывания.

— Выведите результаты на экран.


In [6]:
spark = SparkSession.builder.appName("hw5_spark").getOrCreate()
df = spark.read.option("header",True).csv("book1.csv")
filtered_df = df.filter(df.sales > 3000)
filtered_df.show()
window_spec = Window.partitionBy("genre")
sum_sales_by_genre = fn.sum("sales").over(window_spec)
sum_sales_by_genre_df = df.withColumn("sum_sales_by_genre", sum_sales_by_genre)
sum_sales_by_genre_df.select(["genre", "sum_sales_by_genre"]).distinct() \
    .orderBy(sum_sales_by_genre_df.sum_sales_by_genre.desc()).show()

+--------------------+--------------------+--------------------+-----+------+
|               title|              author|               genre|sales|  year|
+--------------------+--------------------+--------------------+-----+------+
|           """1984""|   ""George Orwell""| ""Science Fiction""| 5000| 1949"|
|"""To Kill a Mock...|      ""Harper Lee""| ""Southern Gothic""| 4000| 1960"|
|"""The Great Gats...| ""F. Scott Fitzg...|           ""Novel""| 4500| 1925"|
+--------------------+--------------------+--------------------+-----+------+

+--------------------+------------------+
|               genre|sum_sales_by_genre|
+--------------------+------------------+
|           ""Novel""|            6500.0|
| ""Science Fiction""|            5000.0|
| ""Southern Gothic""|            4000.0|
|         ""Fantasy""|            3000.0|
+--------------------+------------------+

